# Performance Inflation on Graph-Structured Data

From biology to telecommunications, many real world networks can be described by the process of preferential attachment where well-connected (high *degree*) nodes are more likely to form new edges. In terms of degrees, "the rich get richer".

A diverse set of prediction problems in biology involve interactions between elements of a single set (e.g., proteins), or between elements of two disjoint sets (e.g. protein-ligand, drug-target, enhancer-promoter).  These sets and their interactions can be represented by a graph or network structure, with nodes representing elements of a set and edges representing their interactions.

However, edges that share nodes create non-independent examples when graphs are encoded into a tabular format for machine learning.  This notebook illustrates this encoding process and how the resulting dependent examples can inflate cross-validation performance due to information leakage.  This leakage scales with the number of edges in the graph and affects both linear and non-linear classifiers, though the latter scale more quickly.

The amount of leakage that occurs depends on the dataset, but has impacted multiple independent areas of genomics discussed in our review.  We demonstrate the generality of this pitfall for any graph-structured dataset where non-graph-aware machine learning algorithms are applied.  The pitfall is not specific to graphs formed by preferential attachment, though the presence of high degree nodes in such networks is helpful for demonstrating leakage.

[NetworkX](https://networkx.github.io) is used for the creation and manipulation of graphs.  Nodes are represented by integer identifiers and edges as integer tuples containing source and destination node identifiers.  Nodes and edges may have optional *attributes* which store data as key/value pairs.  No further knowledge of NetworkX should be required to understand the following notebook.

#### - Sean Whalen, Gladstone Institutes, Pollard Lab

In [ ]:
import numpy as np
import pandas as pd
import random

from itertools import chain, product
from joblib import Parallel, delayed
from networkx import nx
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupKFold, StratifiedKFold, cross_val_score

We begin by generating a sequence of numbers drawn from a power law distribution which often captures the long-tailed degree distribution of networks formed by preferential attachment.

In [ ]:
degree_sequence = nx.utils.powerlaw_sequence(8)
print(degree_sequence)

Converted to integers, this degree sequence specifies how many edges each node has.  We will use this sequence to inform the generation of a random graph.

A bipartite graph has nodes consisting of two disjoint sets.  We create a random bipartite graph by specifying the degree sequence of the first set (the length of this sequence specifies the number of nodes), the probability of creating a new node in the second set versus creating a new edge via preferential attachment, and a seed for the random number generator.

In [ ]:
graph = nx.bipartite.preferential_attachment_graph(
    np.asarray(degree_sequence, int),
    p = 0.6,
    seed = 0
)

While creating the graph, this function adds a node *attribute* named `bipartite` that is set to 0 or 1 if the node is in set 1 or 2, respectively.

We can view this graph by determining which nodes are in set 1 and drawing the graph using a bipartite layout.

In [ ]:
set1_nodes = {node for node, data in graph.nodes(data = True) if data['bipartite'] == 0}
layout = nx.bipartite_layout(graph, set1_nodes)
nx.draw(graph, layout)

Note that sets 1 and 2 can have different numbers of nodes, and that nodes can have multiple edges.

A common machine learning task in genomics is link (edge) prediction, where the algorithm learns properties of connected node pairs in one graph to predict links between node pairs in another. Examples include predicting novel protein interactions, or predicting chromatin interactions between enhancers and promoters.  Most machine learning algorithms cannot handle graphs directly, and instead operate on a matrix of features where rows are examples and columns are features describing those examples. This is a *tabular* format, and so we refer to the tabular representation of a graph as its *tabular encoding*.

In a tabular encoding, each node is described by a feature vector and edges (node pairs) are thus the concatenation of two node feature vectors. To create labels for supervised learning, positive examples are node pairs with an edge in the graph, and negative examples are node pairs without edges.  This process is illustrated in a toy graph with disjoint node sets *A* and *B* that could (for example) represent proteins and ligands, drugs and targets, or enhancers and promoters.

![](../images/leakage.png)

Evaluating a model trained on a tabular encoding of a graph using k-fold cross-validation can cause potentially severe performance inflation. This is because node pairs are dependent (violating the Independent-and-Identically-Distributed assumption of most statistical models), and k-fold cross-validation is unaware of these dependencies. In the above example, the edge *A1-B1* is in the training set, and *A1-B2* is in the test set. The model will observe the features of node *A1* during training, and so may be more likely to predict a link between *A1* and *B2* independent of *B2*'s features. In other words, the model may end up learning the degree distribution of specific nodes, rather than properties of the edge that will generalize to unobserved data.

The function below creates a random bipartite graph for a given set of parameters, creates a tabular encoding of that graph, and evaluates the performance of baseline (random), logistic regression, and random forest classifiers using both non-blocking and blocking cross-validation.  Importantly, nodes are assigned random length-3 feature vectors: classifiers should not have above random performance.

For simplicity, blocking cross-validation is passed the source node of each edge so that all edges with the same source node will be either in the training set or the test set.  This will reduce leakage but not eliminate it; in practice, disjoint sets of edges that share no nodes should be identified.  For many genomics problems this is possible by holding out entire chromosomes as test sets.  However, it may not be possible to completely eliminate information leakage for more complex networks.

In [ ]:
def get_performance(total_set1_nodes, bottom_node_probability, seed):
    # initialize random number generators
    random.seed(seed)
    np.random.seed(seed)

    # create a power law degree distribution,
    # then create a random bipartite graph with preferentially attached edges
    degree_sequence = nx.utils.powerlaw_sequence(total_set1_nodes)
    graph = nx.bipartite.preferential_attachment_graph(
        np.asarray(degree_sequence, int),
        p = bottom_node_probability,
        seed = seed
    )

    # compute nodes in sets 1 and 2
    set1_nodes = {node for node, data in graph.nodes(data = True) if data['bipartite'] == 0}
    set2_nodes = graph.nodes - set1_nodes

    # compute missing edges as the set of possible edges minus the set of existing edges
    possible_edges = set(product(set1_nodes, set2_nodes))
    actual_edges = set(graph.edges())
    missing_edges = possible_edges - actual_edges

    # subsample missing edges for use as negative examples
    missing_edges_subset = random.sample(
        list(missing_edges),
        graph.number_of_edges() * negatives_per_positive
    )

    # create random length-3 feature vector for each node in graph
    for node in graph:
        graph.nodes[node]['features'] = np.random.normal(0, 1, 3)

    # featurize all edges (node pairs) for actual and missing edges
    features = [
        np.append(graph.nodes[src]['features'], graph.nodes[dst]['features'])
        for src, dst in list(actual_edges) + list(missing_edges_subset)
    ]
    
    # compute first node of each edge so blocking cv can respect dependencies
    features_set1_nodes = [src for src, dst in list(actual_edges) + list(missing_edges_subset)]
    
    # supervised labels: 1 for positive examples, 0 for negative examples
    labels = np.array([1] * len(actual_edges) + [0] * len(missing_edges_subset))

    # 5-fold cross-validation with shuffling, ignores node dependencies
    cv = StratifiedKFold(
        n_splits = 5,
        shuffle = True,
        random_state = seed
    )
    
    # blocking cross-validation, respects node dependencies
    blocking_cv = GroupKFold(
        n_splits = 5
    )
    
    # compute percent of positive classes in blocking cv test sets, for potential plotting
    class_balances = [
        labels[test_indices].mean()
        for train_indices, test_indices in blocking_cv.split(features, labels, features_set1_nodes)
    ]

    # classifier generating random predictions for baseline performance estimate
    baseline_estimator = DummyClassifier(strategy = 'uniform')
    
    # linear classifier
    lm_estimator = LogisticRegression()
    
    # ensemble classifier
    rf_estimator = RandomForestClassifier(n_estimators = 50, random_state = seed)
    
    # evaluate all models using blocking and non-blocking cv
    estimators = [('baseline', baseline_estimator), ('lm', lm_estimator), ('rf', rf_estimator)]
    all_scores = []

    for model_type, estimator in estimators:
        scores = cross_val_score(
            estimator,
            features,
            labels,
            cv = cv,
            scoring = scoring
        )

        blocked_scores = cross_val_score(
            estimator,
            features,
            labels,
            cv = blocking_cv,
            groups = features_set1_nodes,
            scoring = scoring
        )

        all_scores.append([
            total_set1_nodes,
            graph.number_of_nodes(),
            len(actual_edges),
            bottom_node_probability,
            seed,
            model_type,
            'unblocked',
            np.median(class_balances),
            np.median(scores)
        ])

        all_scores.append([
            total_set1_nodes,
            graph.number_of_nodes(),
            len(actual_edges),
            bottom_node_probability,
            seed,
            model_type,
            'blocked',
            np.median(class_balances),
            np.median(blocked_scores)
        ])

    return all_scores

We now define a scoring function (`average_precision` is area under the precision-recall curve), the number of missing links to featurize per existing link, and a set of parameters for generating random bipartite graphs.  For speed we recommend using the defaults below, though the results have been observed to hold for graphs of many sizes across thousands of initial conditions (seeds).

In [ ]:
scoring = 'average_precision'
negatives_per_positive = 5

params = product(
    [100], # number of nodes
    np.arange(0.3, 0.75, 0.05), # probability of new node vs new preferentially attached edge
    range(5) # seeds
)

Finally, we generate random graphs for all parameters in parallel, using all available CPU cores.  This may take a while on low core system.

In [ ]:
scores = Parallel(-1)(
    delayed(get_performance)(*_)
    for _ in params
)

For plotting, we convert these results to a Pandas dataframe.

In [ ]:
scores = pd.DataFrame(
    chain.from_iterable(scores),
    columns = [
        'total_set1_nodes',
        'total_nodes',
        'total_edges',
        'bottom_node_probability',
        'seed',
        'model_type',
        'cv_type',
        'median_positives_percent',
        'score'
    ]
)

For each combination of model and cross-validation type, we plot a linear model fit to performance as a function of the total number of graph edges.

In [ ]:
import seaborn as sns

sns.set_style('white')

scores['model_and_cv_type'] = scores['model_type'] + '.' + scores['cv_type']

sns.lmplot(
    x = 'total_edges',
    y = 'score',
    hue = 'model_and_cv_type',
    palette = 'Paired',
    data = scores
);

Unblocked cross-validation performance of both models increases with the number of edges, with the random forest more quickly exploiting leakage due to nodes shared by edges in the training and test sets.  When blocking cross-validation is used, both models return to baseline performance.

The amount of edges crossing the train-test divide is particularly problematic in a graph formed via preferential attachment, and so we emphasize the amount of leakage is dependent on the properties of a particular graph-structured dataset.  However, real-world examples in genomics have demonstrated extreme performance inflation where non-blocking cross-validation estimated excellent predictive performance for models that were later shown to capture other properties like degree distribution.  We hope this notebook increases awareness of this pitfall and its prevention.